In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image

In [25]:
from constants import *

In [26]:
import matplotlib.pyplot as plt

### Extracting Nietzche Corpus

In [27]:
from fastai.io import *

In [28]:
PATH='../data/nietzsche/'

In [29]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 600893


In [30]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [31]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


In [32]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [33]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [34]:
idx = [char_indices[c] for c in text]

idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [35]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

In [216]:
def idx2char(idx):
    return ''.join(indices_char[i] for i in idx)

### One hot encoding

In [36]:
def one_hot(a,c): 
    return np.eye(c)[a]

In [152]:
class TextDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, text, idx, vocab_size, timesteps):
        self.vocab_size = vocab_size
        self.text = text
        self.dataset = idx
        self.data_length = len(idx)
        self.timesteps = timesteps

    def __len__(self):
        return (self.data_length // self.timesteps)

    def __getitem__(self, idx):
        start = idx*self.timesteps
        x = self.dataset[start:start+self.timesteps]
        y = self.dataset[start+1:start+self.timesteps+1]
        return one_hot(x, vocab_size), np.array(y)
#         return np.array(x), np.array(y)


In [153]:
batch_size = 64
timesteps = 64
md = TextDataset(text, idx, vocab_size, timesteps)
# md = MusicDataset(h5_file='concat_corpus.h5', set_type='train', json_file='concat_corpus.json', timesteps=timesteps, root_dir=CONCAT_DIR)

In [154]:
train_loader = torch.utils.data.DataLoader(md,
    batch_size=batch_size)

### Dataset sanity test

In [155]:
train_iter = enumerate(train_loader)

In [156]:
i, (x, y) = next(train_iter)
i2, (x2, y2) = next(train_iter)

In [157]:
md.dataset[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [158]:
idx2char(md.dataset[:10])

'PREFACE\n\n\n'

In [161]:
x2[0]


    0     0     0  ...      0     0     0
    0     0     1  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.DoubleTensor of size 64x85]

In [159]:
idx2char(x2)

KeyError: 
    0     0     0  ...      0     0     0
    0     0     1  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.DoubleTensor of size 64x85]


### Model

In [162]:
cuda_enabled = torch.cuda.is_available()

In [163]:
def repackage_var(h):
    """Wraps h in new Variables, to detach them from their history."""
    if type(h) == torch.autograd.Variable:
        v = torch.autograd.Variable(h.data)
        return v.cuda() if cuda_enabled else v
    else:
        return tuple(repackage_var(v) for v in h)

In [83]:
class StatefulLSTM(torch.nn.Module):
    def __init__(self, scale_size, n_hidden, n_factors, bs, nl):
        super().__init__()
        self.scale_size = scale_size
        self.nl = nl
        self.embedding = torch.nn.Embedding(scale_size, n_factors)
        
        self.rnn1 = torch.nn.LSTM(n_factors, n_hidden, nl, dropout=0.5, batch_first=True)
        self.rnn2 = torch.nn.LSTM(n_hidden, n_hidden, nl, dropout=0.5, batch_first=True)
        self.rnn3 = torch.nn.LSTM(n_hidden, n_hidden, nl, dropout=0.5, batch_first=True)
        
        if cuda_enabled:
            self.rnn1 = self.rnn1.cuda()
            self.rnn2 = self.rnn2.cuda()
            self.rnn3 = self.rnn3.cuda()
        
#         self.bn1 = nn.utils.weight_norm(self.rnn1, 'weight_hh_l0')
#         self.bn1 = nn.utils.weight_norm(self.bn1, 'weight_ih_l0')
#         self.bn2 = nn.utils.weight_norm(self.rnn2, 'weight_hh_l0')
#         self.bn2 = nn.utils.weight_norm(self.bn2, 'weight_ih_l0')
#         self.bn3 = nn.utils.weight_norm(self.rnn3, 'weight_hh_l0')
#         self.bn3 = nn.utils.weight_norm(self.bn3, 'weight_ih_l0')
        
        # pytorch rnn does not currently work with batchnorm
        self.l_out = torch.nn.Linear(n_hidden, scale_size)
        self.n_hidden = n_hidden
        self.reset_all_hidden(bs)
        self.bs = bs
        
    def forward(self, notes):
        bs = notes.shape[0]
        if self.h1[0].size(1) != bs: 
            self.reset_all_hidden(bs)
        emb = self.embedding(notes)
        outp1,h1 = self.rnn1(emb, self.h1)
#         outp2,h2 = self.bn2(outp1, self.h2)
#         outp3,h3 = self.bn3(outp2, self.h3)
        self.h1 = repackage_var(h1)
#         self.h2 = repackage_var(h2)
#         self.h3 = repackage_var(h3)
        return torch.nn.functional.log_softmax(self.l_out(outp1), dim=-1).view(-1, self.scale_size)
#         return torch.nn.functional.log_softmax(self.l_out(outp[:, -1, :]), dim=-1)
#         return torch.nn.functional.softmax(self.l_out(outp[:, -1, :]), dim=-1)
    
    def reset_all_hidden(self, bs):
        self.h1 = self.init_hidden(bs)
        self.h2 = self.init_hidden(bs)
        self.h3 = self.init_hidden(bs)
        
    def init_hidden(self, bs):
        h1 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        h2 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        if cuda_enabled:
            return (h1.cuda(), h2.cuda())
        return h1, h2

In [169]:
class StatefulLSTM(torch.nn.Module):
    def __init__(self, scale_size, n_hidden, n_factors, bs, nl):
        super().__init__()
        self.scale_size = scale_size
        self.nl = nl
#         self.embedding = torch.nn.Embedding(scale_size, n_factors)
        
        self.rnn1 = torch.nn.LSTM(scale_size, n_hidden, nl, dropout=0.5, batch_first=True)
        self.rnn2 = torch.nn.LSTM(n_hidden, n_hidden, nl, dropout=0.5, batch_first=True)
        self.rnn3 = torch.nn.LSTM(n_hidden, n_hidden, nl, dropout=0.5, batch_first=True)
        
        if cuda_enabled:
            self.rnn1 = self.rnn1.cuda()
            self.rnn2 = self.rnn2.cuda()
            self.rnn3 = self.rnn3.cuda()
        
#         self.bn1 = nn.utils.weight_norm(self.rnn1, 'weight_hh_l0')
#         self.bn1 = nn.utils.weight_norm(self.bn1, 'weight_ih_l0')
#         self.bn2 = nn.utils.weight_norm(self.rnn2, 'weight_hh_l0')
#         self.bn2 = nn.utils.weight_norm(self.bn2, 'weight_ih_l0')
#         self.bn3 = nn.utils.weight_norm(self.rnn3, 'weight_hh_l0')
#         self.bn3 = nn.utils.weight_norm(self.bn3, 'weight_ih_l0')
        
        # pytorch rnn does not currently work with batchnorm
        self.l_out = torch.nn.Linear(n_hidden, scale_size)
        self.n_hidden = n_hidden
        self.reset_all_hidden(bs)
        self.bs = bs
        
    def forward(self, notes):
        bs = notes.shape[0]
        if self.h1[0].size(1) != bs: 
            self.reset_all_hidden(bs)
#         emb = self.embedding(notes)
        outp1,h1 = self.rnn1(notes, self.h1)
#         outp2,h2 = self.bn2(outp1, self.h2)
#         outp3,h3 = self.bn3(outp2, self.h3)
        self.h1 = repackage_var(h1)
#         self.h2 = repackage_var(h2)
#         self.h3 = repackage_var(h3)
        return torch.nn.functional.log_softmax(self.l_out(outp1), dim=-1).view(-1, self.scale_size)
#         return torch.nn.functional.log_softmax(self.l_out(outp[:, -1, :]), dim=-1)
#         return torch.nn.functional.softmax(self.l_out(outp[:, -1, :]), dim=-1)
    
    def reset_all_hidden(self, bs):
        self.h1 = self.init_hidden(bs)
        self.h2 = self.init_hidden(bs)
        self.h3 = self.init_hidden(bs)
        
    def init_hidden(self, bs):
        h1 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        h2 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        if cuda_enabled:
            return (h1.cuda(), h2.cuda())
        return h1, h2

### Training

In [170]:
m = StatefulLSTM(md.vocab_size, n_hidden=256, n_factors=2, bs=batch_size, nl=2)
if cuda_enabled:
    m = m.cuda()

In [171]:
train_op = torch.optim.Adam(m.parameters(), lr=1e-2)

In [172]:
loss_fn = torch.nn.NLLLoss()

In [174]:
display_step = 100
training_steps = 20
for step in range(training_steps):
# for step in tqdm(range(training_steps)):
    for i, (data,target) in enumerate(train_loader):
        data, target = torch.autograd.Variable(data.float()), torch.autograd.Variable(target.long())
        if cuda_enabled:
            data, target = data.cuda(), target.cuda()
        m.zero_grad()
        forward = m(data)
        loss = loss_fn(forward, target.view(-1))
        loss.backward()
        train_op.step()
        if ((i+1) % display_step == 0):
            print(f'Iteration: {i+1} Loss: {loss.data[0]}')
    print(f'Step: {step} Loss: {loss.data[0]}')

Iteration: 100 Loss: 3.100971221923828
Step: 0 Loss: 6.7294697761535645
Iteration: 100 Loss: 2.9367051124572754
Step: 1 Loss: 4.379326343536377
Iteration: 100 Loss: 2.871424674987793
Step: 2 Loss: 2.55930495262146
Iteration: 100 Loss: 2.4127304553985596
Step: 3 Loss: 2.2250146865844727
Iteration: 100 Loss: 2.1682047843933105
Step: 4 Loss: 2.0256271362304688
Iteration: 100 Loss: 2.02180552482605
Step: 5 Loss: 1.8907009363174438
Iteration: 100 Loss: 1.8985837697982788
Step: 6 Loss: 1.7914042472839355
Iteration: 100 Loss: 1.7919671535491943
Step: 7 Loss: 1.694566011428833
Iteration: 100 Loss: 1.7399513721466064
Step: 8 Loss: 1.6267189979553223
Iteration: 100 Loss: 1.6889568567276
Step: 9 Loss: 1.5837594270706177
Iteration: 100 Loss: 1.6574040651321411
Step: 10 Loss: 1.5568667650222778
Iteration: 100 Loss: 1.6221787929534912
Step: 11 Loss: 1.5133399963378906
Iteration: 100 Loss: 1.5999693870544434
Step: 12 Loss: 1.4856986999511719
Iteration: 100 Loss: 1.5745209455490112
Step: 13 Loss: 1.43

In [65]:
display_step = 100
training_steps = 20
for step in range(training_steps):
# for step in tqdm(range(training_steps)):
    for i, (data,target) in enumerate(train_loader):
        data, target = torch.autograd.Variable(data.long()), torch.autograd.Variable(target.long())
        if cuda_enabled:
            data, target = data.cuda(), target.cuda()
        m.zero_grad()
        forward = m(data)
        loss = loss_fn(forward, target.view(-1))
        loss.backward()
        train_op.step()
        if ((i+1) % display_step == 0):
            print(f'Iteration: {i+1} Loss: {loss.data[0]}')
    print(f'Step: {step} Loss: {loss.data[0]}')

Iteration: 100 Loss: 1.7637908458709717
Step: 0 Loss: 1.6685713529586792


KeyboardInterrupt: 

In [42]:
gen_idxs = generate_sequence(song_seed, 500)

### Saving model

In [27]:
model_path = f'{OUT_DIR}/../models/nietzsche_stackedlstm_rnn_t64.h5'

In [28]:
torch.save(m.state_dict(), model_path)

In [24]:
if cuda_enabled:
    m.load_state_dict(torch.load(model_path))
else:
    m.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage))

### Generate music

Need to have unknown state 0?

In [88]:
timesteps = md.timesteps

In [118]:
list(np.arange(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [218]:
def generate_sequence(song, seq_length):
    full_song = song.tolist()
    # generate music!
#     m.reset_all_hidden(batch_size)
    for i in range(seq_length):
#         seed = np.array([full_song[-timesteps:]])
        seed = np.array([one_hot(full_song[-timesteps:], vocab_size)])
#         print(''.join(indices_char[i] for i in full_song[-timesteps:]))
        # Use our RNN for prediction using our seed! 
        seed_v = torch.autograd.Variable(torch.from_numpy(seed).float())
        if cuda_enabled:
            seed_v = seed_v.cuda()
        predict_probs = m(seed_v)

        # Define output vector for our generated song by sampling from our predicted probability distribution        
        sampled_note = torch.multinomial(predict_probs[-1].exp(), 1).data[0]
#         print('Sampled_note:', sampled_note)
        full_song.append(sampled_note)
        
        # With multi output model, use only the last prediction. As it is predicting to n timesteps
#         v, idx = torch.max(torch.exp(predict_probs[-1]), 0)
#         print(idx)
#         full_song.append(idx.data[0])
    return full_song
    


In [219]:
gen_idxs = generate_sequence(np.array(song_seed), 500)

In [220]:
idx2char(gen_idxs)

'hopenhauer, under the dominion and delusion of\nmorality,--whoever in ard one\'s astemption the matter of\nthe too\nmodelon!=--Their wire himself. As example and some camoring unavounce All little chanced experiences\nwith itself, that the other as it is\nthe\nliveness"\nof the our expise of deligitary morashy in sociom.=--The appearf as upon it him\nwithin an atquined more man would not mather these kind in one\'s belienity of thereby proof analy in\nEncoded\nof his emotion to the heavinaves and impursicnes of Right and\nblind responssible and bath from this necessary,\n'

In [184]:
def get_x_input(partial):
    _, _, _, seq = partial
    input = seq[-timesteps:]
    input_var = torch.autograd.Variable(torch.LongTensor([input]))
    if cuda_enabled:
        input_var = input_var.cuda()
    return input_var

# song = string
# seq_length = generated song length
# beam_size = what to choose from
def beam_search(song, seq_length, beam_size):    
    full_song = song.tolist()
    m.reset_all_hidden(batch_size)
    partial_sequences = [(0, 0, [], full_song)]
    m.eval()

    for i in range(seq_length):
        partial_sequences = find_partials(partial_sequences, beam_size)
        
    final_sequence = partial_sequences[0][3]
    return final_sequence
    
def find_partials(partial_sequences, beam_size):
    partial_next = []
    for partial in partial_sequences:
        it, tot_p, p_list, seq = partial
        x_input = get_x_input(partial)

        predict_probs = m(x_input)
        # last_it_probs = torch.exp(predict_probs[-(it+1):]) # this is to predict the last few iterations
        last_it_probs = torch.exp(predict_probs[-1:])
        top, idxs = torch.topk(last_it_probs, beam_size, 1)

        for i in range(beam_size):
            prob = top.data[0][i]
            idx = idxs.data[0][i]
            new_p_list = p_list+[prob]
            partial_next.append((it+1, np.mean(new_p_list), new_p_list, seq+[idx]))

    partial_sequences = sorted(partial_next, key=lambda x: x[1], reverse=True)[:3]
    return partial_sequences

In [91]:
def random_choice(top, idxs):
    return np.random.choice(
      idxs.data.numpy().reshape(-1), 
      1,
      p=(top/top.sum()).data.numpy().reshape(-1)
    )

153093

In [105]:
random_seed = random.randint(0, len(md.dataset)//2)
song_seed = md.dataset[random_seed:random_seed+md.timesteps]
# generated_idxs = generate_sequence(song_seed, 500)

In [175]:
''.join(indices_char[i] for i in song_seed)

'hopenhauer, under the dominion and delusion of\nmorality,--whoeve'

In [ ]:
bs_gen_idxs = beam_search(np.array(song_seed), 500, 3)

In [187]:
gen_idxs = generate_sequence(np.array(song_seed), 500)

In [195]:
idx2char(gen_idxs)

'hopenhauer, under the dominion and delusion of\nmorality,--whoever the '

In [196]:
idx2char(bs_gen_idxs)

'hopenhauer, under the dominion and delusion of\nmorality,--whoever, tha'

### Beam search end - Decoding time

In [38]:
import decode

In [39]:
def decode_output(output_idx):
    idx2token = md.concat_json['idx_to_token']
    token_list = list(map(lambda x: idx2token.get(str(x), ''), output_idx))
    return decode_token(token_list)

def decode_token(token_list):
    if (token_list[0] != START_DELIM):
        token_list.insert(0, START_DELIM)
    token_str = ''.join(token_list)
    with open(f'{SCRATCH_DIR}/utf_to_txt.json', 'r') as f:
        utf_to_txt = json.load(f)
    score, stream = decode.decode_string(utf_to_txt, token_str)
    return token_str, score, stream

# test = [idx2token[f'{x}'] for x in seq_arr]; test

In [52]:
song_seed = md.dataset[:md.timesteps]
# generated_idxs = generate_sequence(song_seed, 500)

In [54]:
bs_gen_idxs = beam_search(np.array(song_seed), 500, 3)

In [56]:
''.join(indices_char[i] for i in bs_gen_idxs)

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the present of their in the present of the sense of t'

In [42]:
gen_idxs = generate_sequence(song_seed, 500)